In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1063527,2021-03-07,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1063528,2021-03-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1063529,2021-03-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1063530,2021-03-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28763,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28765,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28767,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28769,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28771,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
657931,2021-03-07,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804
657933,2021-03-08,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804
657935,2021-03-09,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804
657937,2021-03-10,Arkansas,Arkansas,5001,2006,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1063527,2021-03-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1063528,2021-03-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1063529,2021-03-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1063530,2021-03-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-10') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
414,2021-03-10,Snohomish,Washington,31510,548.00,53061,WA,County,822083,3832.95,66.66
827,2021-03-10,Cook,Illinois,480487,10020.00,17031,IL,County,5150233,9329.42,194.55
1239,2021-03-10,Orange,California,263111,4346.00,6059,CA,County,3175692,8285.15,136.85
1650,2021-03-10,Maricopa,Arizona,517726,9368.00,4013,AZ,County,4485414,11542.44,208.85
2061,2021-03-10,Los Angeles,California,1207361,22213.00,6037,CA,County,10039107,12026.58,221.26
...,...,...,...,...,...,...,...,...,...,...,...
1063054,2021-03-10,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1063204,2021-03-10,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1063323,2021-03-10,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1063438,2021-03-10,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
414,2021-03-10,Snohomish,Washington,31510,548.00,53061,WA,County,822083,3832.95,66.66,66.66,3832.95
827,2021-03-10,Cook,Illinois,480487,10020.00,17031,IL,County,5150233,9329.42,194.55,194.55,9329.42
1239,2021-03-10,Orange,California,263111,4346.00,6059,CA,County,3175692,8285.15,136.85,136.85,8285.15
1650,2021-03-10,Maricopa,Arizona,517726,9368.00,4013,AZ,County,4485414,11542.44,208.85,208.85,11542.44
2061,2021-03-10,Los Angeles,California,1207361,22213.00,6037,CA,County,10039107,12026.58,221.26,221.26,12026.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063054,2021-03-10,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1063204,2021-03-10,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1063323,2021-03-10,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1063438,2021-03-10,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
748508,2021-03-10,Clarke,Alabama,3449,53.00,1025,AL,County,23622,14600.80,224.37,224.37,14600.80,1
687863,2021-03-10,Hale,Alabama,2130,71.00,1065,AL,County,14651,14538.26,484.61,484.61,14538.26,2
538580,2021-03-10,Lowndes,Alabama,1338,51.00,1085,AL,County,9726,13756.94,524.37,524.37,13756.94,3
477690,2021-03-10,Etowah,Alabama,13454,329.00,1055,AL,County,102268,13155.63,321.70,321.70,13155.63,4
538931,2021-03-10,Marengo,Alabama,2457,58.00,1091,AL,County,18863,13025.50,307.48,307.48,13025.50,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656545,2021-03-10,Converse,Wyoming,972,17.00,56009,WY,County,13822,7032.27,122.99,122.99,7032.27,19
656894,2021-03-10,Sublette,Wyoming,681,7.00,56035,WY,County,9831,6927.07,71.20,71.20,6927.07,20
815590,2021-03-10,Lincoln,Wyoming,1275,12.00,56023,WY,County,19830,6429.65,60.51,60.51,6429.65,21
907206,2021-03-10,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
538580,2021-03-10,Lowndes,Alabama,1338,51.00,1085,AL,County,9726,13756.94,524.37,524.37,13756.94,3,1
687863,2021-03-10,Hale,Alabama,2130,71.00,1065,AL,County,14651,14538.26,484.61,484.61,14538.26,2,2
229499,2021-03-10,Walker,Alabama,6636,264.00,1127,AL,County,63521,10446.94,415.61,415.61,10446.94,31,3
476282,2021-03-10,Clay,Alabama,1465,54.00,1027,AL,County,13235,11069.13,408.01,408.01,11069.13,20,4
537878,2021-03-10,Crenshaw,Alabama,1476,55.00,1041,AL,County,13772,10717.40,399.36,399.36,10717.40,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579296,2021-03-10,Hot Springs,Wyoming,362,3.00,56017,WY,County,4413,8203.04,67.98,67.98,8203.04,17,19
815590,2021-03-10,Lincoln,Wyoming,1275,12.00,56023,WY,County,19830,6429.65,60.51,60.51,6429.65,21,20
795353,2021-03-10,Uinta,Wyoming,2084,12.00,56041,WY,County,20226,10303.57,59.33,59.33,10303.57,4,21
228425,2021-03-10,Teton,Wyoming,3404,9.00,56039,WY,County,23464,14507.33,38.36,38.36,14507.33,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3832.95,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3832.95,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3832.95,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3832.95,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3832.95,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061021,2021-03-07,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1061022,2021-03-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1061023,2021-03-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1061024,2021-03-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
745659,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14600.80,40,1,2.00
745660,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,224.37,14600.80,40,1,0.00
745661,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,224.37,14600.80,40,1,1.00
745662,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,224.37,14600.80,40,1,4.00
745663,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,224.37,14600.80,40,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829155,2021-03-07,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
829156,2021-03-08,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
829157,2021-03-09,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
829158,2021-03-10,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
536423,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13756.94,1,3,1.00,0.00
536424,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13756.94,1,3,0.00,0.00
536425,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13756.94,1,3,0.00,0.00
536426,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13756.94,1,3,0.00,0.00
536427,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13756.94,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534663,2021-03-07,Albany,Wyoming,3913,11.00,56001,WY,County,38880,10064.30,28.29,28.29,10082.30,23,6,0.00,0.00
534664,2021-03-08,Albany,Wyoming,3917,11.00,56001,WY,County,38880,10074.59,28.29,28.29,10082.30,23,6,4.00,0.00
534665,2021-03-09,Albany,Wyoming,3920,11.00,56001,WY,County,38880,10082.30,28.29,28.29,10082.30,23,6,3.00,0.00
534666,2021-03-10,Albany,Wyoming,3920,11.00,56001,WY,County,38880,10082.30,28.29,28.29,10082.30,23,6,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-10') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
273859,2021-03-10,Imperial,California,27045,663.00,6025,CA,County,181215,14924.26,365.86,365.86,14924.26,1,2,12.00,5.00
2061,2021-03-10,Los Angeles,California,1207361,22213.00,6037,CA,County,10039107,12026.58,221.26,221.26,12026.58,2,5,1437.00,114.00
479050,2021-03-10,Inyo,California,1324,37.00,6027,CA,County,18039,7339.65,205.11,205.11,7339.65,3,25,4.00,0.00
61160,2021-03-10,Stanislaus,California,57124,960.00,6099,CA,County,550660,10373.73,174.34,174.34,10373.73,4,10,139.00,1.00
75797,2021-03-10,Tulare,California,48432,781.00,6107,CA,County,466195,10388.79,167.53,167.53,10388.79,5,9,33.00,5.00
28391,2021-03-10,Riverside,California,291675,3941.00,6065,CA,County,2470546,11806.09,159.52,159.52,11806.09,6,6,183.00,1.00
48684,2021-03-10,San Joaquin,California,67754,1208.00,6077,CA,County,762148,8889.87,158.50,158.50,8889.87,7,18,97.00,9.00
27651,2021-03-10,Fresno,California,96744,1509.00,6019,CA,County,999101,9683.11,151.04,151.04,9683.11,8,14,79.00,4.00
434236,2021-03-10,Merced,California,29643,417.00,6047,CA,County,277680,10675.24,150.17,150.17,10675.24,9,8,43.00,1.00
580639,2021-03-10,Kings,California,22250,229.00,6031,CA,County,152940,14548.19,149.73,149.73,14548.19,10,3,24.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1010184,2021-03-10,Lassen,California,5629,24.00,6035,CA,County,30573,18411.67,78.50,78.50,18411.67,30,1,0.00,-1.00
273859,2021-03-10,Imperial,California,27045,663.00,6025,CA,County,181215,14924.26,365.86,365.86,14924.26,1,2,12.00,5.00
580639,2021-03-10,Kings,California,22250,229.00,6031,CA,County,152940,14548.19,149.73,149.73,14548.19,10,3,24.00,0.00
136561,2021-03-10,San Bernardino,California,288135,3198.00,6071,CA,County,2180085,13216.69,146.69,146.69,13216.69,11,4,90.00,18.00
2061,2021-03-10,Los Angeles,California,1207361,22213.00,6037,CA,County,10039107,12026.58,221.26,221.26,12026.58,2,5,1437.00,114.00
28391,2021-03-10,Riverside,California,291675,3941.00,6065,CA,County,2470546,11806.09,159.52,159.52,11806.09,6,6,183.00,1.00
171620,2021-03-10,Kern,California,104509,948.00,6029,CA,County,900202,11609.51,105.31,105.31,11609.51,17,7,66.00,6.00
434236,2021-03-10,Merced,California,29643,417.00,6047,CA,County,277680,10675.24,150.17,150.17,10675.24,9,8,43.00,1.00
75797,2021-03-10,Tulare,California,48432,781.00,6107,CA,County,466195,10388.79,167.53,167.53,10388.79,5,9,33.00,5.00
61160,2021-03-10,Stanislaus,California,57124,960.00,6099,CA,County,550660,10373.73,174.34,174.34,10373.73,4,10,139.00,1.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
273859,2021-03-10,Imperial,California,27045,663.00,6025,CA,County,181215,14924.26,365.86,365.86,14924.26,1,2,12.00,5.00
2061,2021-03-10,Los Angeles,California,1207361,22213.00,6037,CA,County,10039107,12026.58,221.26,221.26,12026.58,2,5,1437.00,114.00
479050,2021-03-10,Inyo,California,1324,37.00,6027,CA,County,18039,7339.65,205.11,205.11,7339.65,3,25,4.00,0.00
61160,2021-03-10,Stanislaus,California,57124,960.00,6099,CA,County,550660,10373.73,174.34,174.34,10373.73,4,10,139.00,1.00
75797,2021-03-10,Tulare,California,48432,781.00,6107,CA,County,466195,10388.79,167.53,167.53,10388.79,5,9,33.00,5.00
28391,2021-03-10,Riverside,California,291675,3941.00,6065,CA,County,2470546,11806.09,159.52,159.52,11806.09,6,6,183.00,1.00
48684,2021-03-10,San Joaquin,California,67754,1208.00,6077,CA,County,762148,8889.87,158.50,158.50,8889.87,7,18,97.00,9.00
27651,2021-03-10,Fresno,California,96744,1509.00,6019,CA,County,999101,9683.11,151.04,151.04,9683.11,8,14,79.00,4.00
434236,2021-03-10,Merced,California,29643,417.00,6047,CA,County,277680,10675.24,150.17,150.17,10675.24,9,8,43.00,1.00
580639,2021-03-10,Kings,California,22250,229.00,6031,CA,County,152940,14548.19,149.73,149.73,14548.19,10,3,24.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,365.86,14924.26,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3582,03/07/21,Lassen,5623,24.00,18392.05,78.50,78.50,18411.67,30,1,0.00,0.00
3583,03/08/21,Lassen,5629,25.00,18411.67,81.77,78.50,18411.67,30,1,6.00,1.00
3584,03/09/21,Lassen,5629,25.00,18411.67,81.77,78.50,18411.67,30,1,0.00,0.00
3585,03/10/21,Lassen,5629,24.00,18411.67,78.50,78.50,18411.67,30,1,0.00,-1.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,365.86,14924.26,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,365.86,14924.26,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3656,03/07/21,Kings,22184,227.00,14505.03,148.42,149.73,14548.19,10,3,9.00,0.00
3657,03/08/21,Kings,22211,229.00,14522.69,149.73,149.73,14548.19,10,3,27.00,2.00
3658,03/09/21,Kings,22226,229.00,14532.50,149.73,149.73,14548.19,10,3,15.00,0.00
3659,03/10/21,Kings,22250,229.00,14548.19,149.73,149.73,14548.19,10,3,24.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)